In [1]:
import pygrib
import numpy as np
import pickle
import os
from collections import OrderedDict
import zipfile

In [29]:
Time = []
try:
    os.mkdir('US_wind')
except:
    pass

for rawfname in os.listdir(os.getcwd() + '/raw'):
    print(rawfname)
    grbs = pygrib.open(os.getcwd() + '/raw/' + rawfname)
    uin = grbs.select(shortName='u', typeOfLevel='isobaricInhPa', level = lambda l: l >= 100 and l <= 1000)
    vin = grbs.select(shortName='v', typeOfLevel='isobaricInhPa', level = lambda l: l >= 100 and l <= 1000)
    grbs.close()

    for i in range(len(uin)):
        u_wind_i = uin[i]
        v_wind_i = vin[i]

        lat_grid = u_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[1].flatten()
        lon_grid = u_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[2].flatten()
        lon_lat_grid = np.dstack((lon_grid, lat_grid))[0]

        pickle.dump(lon_lat_grid, open('LonLat_Grid.p','wb'), protocol=2)

        date_time = str(u_wind_i.year) + '_' + str(u_wind_i.month).zfill(2) + '_' + str(u_wind_i.day).zfill(2) + '_' + str(u_wind_i.hour).zfill(2) + '00'
        if date_time not in Time:

            wind_data = OrderedDict()
            Time.append(date_time)

            wind_data[u_wind_i.level] = np.dstack((u_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[0].flatten(),
                                                     v_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[0].flatten()))[0]

        else:
            wind_data[u_wind_i.level] = np.dstack((u_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[0].flatten(),
                                                     v_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[0].flatten()))[0]

        pickle.dump(wind_data, open(os.getcwd() + '/US_wind/' + date_time + '.p','wb'), protocol=2) # m/s

A24522-201310.pgb.f00
A24351-201301.pgb.f00
A24560-201312.pgb.f00
A24370-201302.pgb.f00
A24389-201303.pgb.f00
A24446-201306.pgb.f00
A24541-201311.pgb.f00
A24503-201309.pgb.f00
A24408-201304.pgb.f00
A24484-201308.pgb.f00
A24465-201307.pgb.f00
A24427-201305.pgb.f00


In [35]:
with zipfile.ZipFile('NCAR_wind.zip', mode='w', compression=zipfile.ZIP_DEFLATED) as zf:
    for fname in os.listdir(os.getcwd() + '/US_wind/'):
        zf.write(os.getcwd() + '/US_wind/' + fname, os.path.basename(os.getcwd() + '/US_wind/' + fname))

In [36]:
with zipfile.ZipFile('NCAR_wind.zip', 'r') as zf:
    flist = zf.namelist()

In [37]:
flist

['2013_10_10_1800.p',
 '2013_06_03_0000.p',
 '2013_12_24_0600.p',
 '2013_08_12_1800.p',
 '2013_01_14_0000.p',
 '2013_10_28_0600.p',
 '2013_08_22_1800.p',
 '2013_09_14_0000.p',
 '2013_06_29_0000.p',
 '2013_06_10_1200.p',
 '2013_12_25_0600.p',
 '2013_01_22_0600.p',
 '2013_04_21_1200.p',
 '2013_08_14_0000.p',
 '2013_03_10_0000.p',
 '2013_03_25_0600.p',
 '2013_06_15_0000.p',
 '2013_07_23_1200.p',
 '2013_08_21_1200.p',
 '2013_02_10_1200.p',
 '2013_10_11_0000.p',
 '2013_12_26_0000.p',
 '2013_06_28_0000.p',
 '2013_11_04_1800.p',
 '2013_06_28_1800.p',
 '2013_12_09_0000.p',
 '2013_01_10_1800.p',
 '2013_01_11_0000.p',
 '2013_11_21_1800.p',
 '2013_05_17_0600.p',
 '2013_02_28_1800.p',
 '2013_08_27_0600.p',
 '2013_09_06_1800.p',
 '2013_08_20_1200.p',
 '2013_10_18_1200.p',
 '2013_03_24_1200.p',
 '2013_08_10_0000.p',
 '2013_04_05_1800.p',
 '2013_02_17_1800.p',
 '2013_06_17_1800.p',
 '2013_05_27_1800.p',
 '2013_04_03_0000.p',
 '2013_11_17_0600.p',
 '2013_02_03_0600.p',
 '2013_10_12_0000.p',
 '2013_09_